In [1]:
!git clone https://github.com/TaoRuijie/TalkNet-ASD.git



Cloning into 'TalkNet-ASD'...
remote: Enumerating objects: 187, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 187 (delta 39), reused 35 (delta 35), pack-reused 133 (from 1)
Receiving objects: 100% (187/187), 53.52 MiB | 31.24 MiB/s, done.
Resolving deltas: 100% (92/92), done.


In [2]:
cd TalkNet-ASD

/kaggle/working/TalkNet-ASD


In [3]:
 !pip install -r requirement.txt



  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.0/122.0 kB 3.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 35.8 MB/s eta 0:00:0000:01
  Created wheel for python_speech_features: filename=python_speech_features-0.6-py3-none-any.whl size=5868 sha256=d43c0122f03c0fa3654852d160aff28eadbe76c05aab492596135f03fa2d8cc5
  Stored in directory: /root/.cache/pip/wheels/5a/9e/68/30bad9462b3926c29e315df16b562216d12bdc215f4d240294
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6082 sha256=3c75b97626bebd2b580804ec340adf601074da6568c4a64cefa893bd1a0abce8
  Stored in directory: /root/.cache/pip/wheels/8e/7a/69/cd6aeb83b126a7f04cbe7c9d929028dc52a6e7d525ff56003a
Successfully built python_speech_features ffmpeg


In [4]:
 !pip install youtube-dl


In [5]:
!pip install yt-dlp


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.3/171.3 kB 3.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 49.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.1/164.1 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 62.7 MB/s eta 0:00:00:00:01
  Attempting uninstall: websockets
    Found existing installation: websockets 12.0
    Uninstalling websockets-12.0:
      Successfully uninstalled websockets-12.0


In [6]:
# New code to change the box utils  to new_box_utils.py
new_code = """\
import numpy as np
from itertools import product as product
import torch
from torch.autograd import Function

def nms_(dets, thresh):
    \"\"\"Courtesy of Ross Girshick
    [https://github.com/rbgirshick/py-faster-rcnn/blob/master/lib/nms/py_cpu_nms.py]
    \"\"\"
    x1 = dets[:, 0]
    y1 = dets[:, 1]
    x2 = dets[:, 2]
    y2 = dets[:, 3]
    scores = dets[:, 4]

    areas = (x2 - x1) * (y2 - y1)
    order = scores.argsort()[::-1]

    keep = []
    while order.size > 0:
        i = order[0]
        keep.append(int(i))
        xx1 = np.maximum(x1[i], x1[order[1:]])
        yy1 = np.maximum(y1[i], y1[order[1:]])
        xx2 = np.minimum(x2[i], x2[order[1:]])
        yy2 = np.minimum(y2[i], y2[order[1:]])

        w = np.maximum(0.0, xx2 - xx1)
        h = np.maximum(0.0, yy2 - yy1)
        inter = w * h
        ovr = inter / (areas[i] + areas[order[1:]] - inter)

        inds = np.where(ovr <= thresh)[0]
        order = order[inds + 1]

    return np.array(keep).astype(int)


def decode(loc, priors, variances):
    \"\"\"Decode locations from predictions using priors to undo
    the encoding we did for offset regression at train time.
    Args:
        loc (tensor): location predictions for loc layers,
            Shape: [num_priors,4]
        priors (tensor): Prior boxes in center-offset form.
            Shape: [num_priors,4].
        variances: (list[float]) Variances of priorboxes
    Return:
        decoded bounding box predictions
    \"\"\"

    boxes = torch.cat((
        priors[:, :2] + loc[:, :2] * variances[0] * priors[:, 2:],
        priors[:, 2:] * torch.exp(loc[:, 2:] * variances[1])), 1)
    boxes[:, :2] -= boxes[:, 2:] / 2
    boxes[:, 2:] += boxes[:, :2]
    return boxes


def nms(boxes, scores, overlap=0.5, top_k=200):
    \"\"\"Apply non-maximum suppression at test time to avoid detecting too many
    overlapping bounding boxes for a given object.
    Args:
        boxes: (tensor) The location preds for the img, Shape: [num_priors,4].
        scores: (tensor) The class predscores for the img, Shape:[num_priors].
        overlap: (float) The overlap thresh for suppressing unnecessary boxes.
        top_k: (int) The Maximum number of box preds to consider.
    Return:
        The indices of the kept boxes with respect to num_priors.
    \"\"\"

    keep = scores.new(scores.size(0)).zero_().long()
    if boxes.numel() == 0:
        return keep, 0
    x1 = boxes[:, 0]
    y1 = boxes[:, 1]
    x2 = boxes[:, 2]
    y2 = boxes[:, 3]
    area = torch.mul(x2 - x1, y2 - y1)
    v, idx = scores.sort(0)  # sort in ascending order
    idx = idx[-top_k:]  # indices of the top-k largest vals
    xx1 = boxes.new()
    yy1 = boxes.new()
    xx2 = boxes.new()
    yy2 = boxes.new()
    w = boxes.new()
    h = boxes.new()

    count = 0
    while idx.numel() > 0:
        i = idx[-1]  # index of current largest val
        keep[count] = i
        count += 1
        if idx.size(0) == 1:
            break
        idx = idx[:-1]  # remove kept element from view
        # load bboxes of next highest vals
        torch.index_select(x1, 0, idx, out=xx1)
        torch.index_select(y1, 0, idx, out=yy1)
        torch.index_select(x2, 0, idx, out=xx2)
        torch.index_select(y2, 0, idx, out=yy2)
        # store element-wise max with next highest score
        xx1 = torch.clamp(xx1, min=x1[i])
        yy1 = torch.clamp(yy1, min=y1[i])
        xx2 = torch.clamp(xx2, max=x2[i])
        yy2 = torch.clamp(yy2, max=y2[i])
        w.resize_as_(xx2)
        h.resize_as_(yy2)
        w = xx2 - xx1
        h = yy2 - yy1
        w = torch.clamp(w, min=0.0)
        h = torch.clamp(h, min=0.0)
        inter = w * h
        rem_areas = torch.index_select(area, 0, idx)  # load remaining areas
        union = (rem_areas - inter) + area[i]
        IoU = inter / union  # store result in IoU
        idx = idx[IoU.le(overlap)]
    return keep, count


class Detect(object):

    def __init__(self, num_classes=2,
                    top_k=750, nms_thresh=0.3, conf_thresh=0.05,
                    variance=[0.1, 0.2], nms_top_k=5000):
        
        self.num_classes = num_classes
        self.top_k = top_k
        self.nms_thresh = nms_thresh
        self.conf_thresh = conf_thresh
        self.variance = variance
        self.nms_top_k = nms_top_k

    def forward(self, loc_data, conf_data, prior_data):

        num = loc_data.size(0)
        num_priors = prior_data.size(0)

        conf_preds = conf_data.view(num, num_priors, self.num_classes).transpose(2, 1)
        batch_priors = prior_data.view(-1, num_priors, 4).expand(num, num_priors, 4)
        batch_priors = batch_priors.contiguous().view(-1, 4)

        decoded_boxes = decode(loc_data.view(-1, 4), batch_priors, self.variance)
        decoded_boxes = decoded_boxes.view(num, num_priors, 4)

        output = torch.zeros(num, self.num_classes, self.top_k, 5)

        for i in range(num):
            boxes = decoded_boxes[i].clone()
            conf_scores = conf_preds[i].clone()

            for cl in range(1, self.num_classes):
                c_mask = conf_scores[cl].gt(self.conf_thresh)
                scores = conf_scores[cl][c_mask]
                
                if scores.dim() == 0:
                    continue
                l_mask = c_mask.unsqueeze(1).expand_as(boxes)
                boxes_ = boxes[l_mask].view(-1, 4)
                ids, count = nms(boxes_, scores, self.nms_thresh, self.nms_top_k)
                count = count if count < self.top_k else self.top_k

                output[i, cl, :count] = torch.cat((scores[ids[:count]].unsqueeze(1), boxes_[ids[:count]]), 1)

        return output


class PriorBox(object):

    def __init__(self, input_size, feature_maps,
                    variance=[0.1, 0.2],
                    min_sizes=[16, 32, 64, 128, 256, 512],
                    steps=[4, 8, 16, 32, 64, 128],
                    clip=False):

        super(PriorBox, self).__init__()

        self.imh = input_size[0]
        self.imw = input_size[1]
        self.feature_maps = feature_maps

        self.variance = variance
        self.min_sizes = min_sizes
        self.steps = steps
        self.clip = clip

    def forward(self):
        mean = []
        for k, fmap in enumerate(self.feature_maps):
            feath = fmap[0]
            featw = fmap[1]
            for i, j in product(range(feath), range(featw)):
                f_kw = self.imw / self.steps[k]
                f_kh = self.imh / self.steps[k]

                cx = (j + 0.5) / f_kw
                cy = (i + 0.5) / f_kh

                s_kw = self.min_sizes[k] / self.imw
                s_kh = self.min_sizes[k] / self.imh

                mean += [cx, cy, s_kw, s_kh]

        output = torch.FloatTensor(mean).view(-1, 4)
        
        if self.clip:
            output.clamp_(max=1, min=0)
        
        return output
"""

# Path to the existing box_utils.py file
file_path = '/kaggle/working/TalkNet-ASD/model/faceDetector/s3fd/box_utils.py'

# Write the new code to the file, replacing the old content
with open(file_path, 'w') as file:
    file.write(new_code)

print(f'Updated {file_path} with new code.')


Updated /kaggle/working/TalkNet-ASD/model/faceDetector/s3fd/box_utils.py with new code.


In [7]:
# Read and verify the contents of the updated file
with open(file_path, 'r') as file:
    updated_content = file.read()
    #print(updated_content);


In [ ]:
# Download the YouTube Video Using yt-dlp: Use the following command to download the video:
!yt-dlp -f best -o 'input_video.mp4' https://www.youtube.com/watch?v=HEAZF_aUkRU

In [8]:
 # Download the YouTube Video Using yt-dlp: Use the following command to download the video:
!yt-dlp -f best -o 'youtube_video.mp4' https://www.youtube.com/watch?v=by0zFH_QIJU



         To let yt-dlp download and merge the best available formats, simply do not pass any format selection.
         If you know what you are doing and want only the best pre-merged format, use "-f b" instead to suppress this warning
[youtube] Extracting URL: https://www.youtube.com/watch?v=by0zFH_QIJU
[youtube] by0zFH_QIJU: Downloading webpage
[youtube] by0zFH_QIJU: Downloading ios player API JSON
[youtube] by0zFH_QIJU: Downloading mweb player API JSON
[youtube] by0zFH_QIJU: Downloading player a4d559ec
[youtube] by0zFH_QIJU: Downloading m3u8 information
[info] by0zFH_QIJU: Downloading 1 format(s): 18
[download] Destination: youtube_video.mp4
[download] 100% of   58.69MiB in 00:00:14 at 4.09MiB/s0;33m00:000m


In [9]:
mv youtube_video.mp4 demo/youtube_video.mp4


In [10]:
!ls demo/

001.avi  001_result.avi  youtube_video.mp4


In [14]:
!ffmpeg -i demo/youtube_video.mp4 -ss 00:10:48 -to 00:12:10 -c copy demo/upd_trimmed_video.mp4


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [15]:
ls demo/

001.avi*         trimmed_video.mp4      youtube_video.mp4
001_result.avi*  upd_trimmed_video.mp4


In [16]:
import torch
print(torch.cuda.is_available())  # This should return True if a GPU is available.


True


In [17]:
!python demoTalkNet.py --videoName upd_trimmed_video.mp4


/opt/conda/lib/python3.10/site-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1KafnHz7ccT-3IyddBsL5yi2xGtxAKypt
To: /kaggle/working/TalkNet-ASD/model/faceDetector/s3fd/sfd_face.pth
100%|███████████████████████████████████████| 89.8M/89.8M [00:00<00:00, 108MB/s]
/opt/conda/lib/python3.10/site-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1AbN9fCf9IexMxEKXLQY2KYBlb-IhSEea
From (redirected): https://drive.google.com/uc?id=1AbN9fCf9IexMxEKXLQY2KYBlb-IhSEea&confirm=t&uuid=6ac9f1cc-2e0e-4b9c-abd2-ad33e72d03b5
To: /kaggle/working/TalkNet-ASD/pretrain_TalkSet.model
100%|█████████████████████████████

In [19]:
#using the active speaker detection model from TalkNet-ASD
!python demoTalkNet.py --videoName upd_trimmed_video --videoFolder /kaggle/working/TalkNet-ASD/demo --pretrainModel pretrain_TalkSet.model


2024-10-11 05:56:44 Extract the video and save in /kaggle/working/TalkNet-ASD/demo/upd_trimmed_video/pyavi/video.avi 
2024-10-11 05:56:44 Extract the audio and save in /kaggle/working/TalkNet-ASD/demo/upd_trimmed_video/pyavi/audio.wav 
2024-10-11 05:56:47 Extract the frames and save in /kaggle/working/TalkNet-ASD/demo/upd_trimmed_video/pyframes 
/kaggle/working/TalkNet-ASD/demo/upd_trimmed_video/pyavi/video.avi - scenes detected 5
2024-10-11 05:56:52 Scene detection and save in /kaggle/working/TalkNet-ASD/demo/upd_trimmed_video/pywork 
2024-10-11 05:57:29 Face detection and save in /kaggle/working/TalkNet-ASD/demo/upd_trimmed_video/pywork 
2024-10-11 05:57:29 Face track and detected 12 tracks 
100%|███████████████████████████████████████████| 12/12 [00:10<00:00,  1.14it/s]
2024-10-11 05:57:39 Face Crop and saved in /kaggle/working/TalkNet-ASD/demo/upd_trimmed_video/pycrop tracks 
10-11 05:57:40 Model para number = 15.01
Model pretrain_TalkSet.model loaded from previous state! 
100%|███

In [21]:
import pickle

# Define the paths to the pickle files
scene_pkl_path = "/kaggle/working/TalkNet-ASD/demo/upd_trimmed_video/pywork/scene.pckl"
tracks_pkl_path = "/kaggle/working/TalkNet-ASD/demo/upd_trimmed_video/pywork/tracks.pckl"
faces_pkl_path = "/kaggle/working/TalkNet-ASD/demo/upd_trimmed_video/pywork/faces.pckl"
scores_pkl_path = "/kaggle/working/TalkNet-ASD/demo/upd_trimmed_video/pywork/scores.pckl"

# Load the pickle files
with open(scene_pkl_path, 'rb') as f:
    scene_data = pickle.load(f)

with open(tracks_pkl_path, 'rb') as f:
    tracks_data = pickle.load(f)

with open(faces_pkl_path, 'rb') as f:
    faces_data = pickle.load(f)

with open(scores_pkl_path, 'rb') as f:
    scores_data = pickle.load(f)

# Inspect the loaded data
print("Scene Data:", type(scene_data), len(scene_data))
print("Tracks Data:", type(tracks_data), len(tracks_data))
print("Faces Data:", type(faces_data), len(faces_data))
print("Scores Data:", type(scores_data), len(scores_data))

# Inspect a sample of each data type for consistency
print("Sample Scene Data:", scene_data[:5])
print("Sample Tracks Data:", tracks_data[:5])
print("Sample Faces Data:", faces_data[:5])
print("Sample Scores Data:", scores_data[:5])


Scene Data: <class 'list'> 5
Tracks Data: <class 'list'> 12
Faces Data: <class 'list'> 2054
Scores Data: <class 'list'> 12
Sample Scene Data: [(00:00:00.000 [frame=0, fps=25.000], 00:00:04.000 [frame=100, fps=25.000]), (00:00:04.000 [frame=100, fps=25.000], 00:01:07.760 [frame=1694, fps=25.000]), (00:01:07.760 [frame=1694, fps=25.000], 00:01:14.920 [frame=1873, fps=25.000]), (00:01:14.920 [frame=1873, fps=25.000], 00:01:18.960 [frame=1974, fps=25.000]), (00:01:18.960 [frame=1974, fps=25.000], 00:01:21.160 [frame=2029, fps=25.000])]
Sample Tracks Data: [{'track': {'frame': array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89, 90, 91, 92, 93, 94,

In [22]:
# Check if all frames have valid tracking and scoring data
num_frames = len(tracks_data)
valid_frames = True

for i in range(num_frames):
    if 'bbox' not in tracks_data[i]['track']:
        print(f"Missing bbox in tracking data for frame {i}")
        valid_frames = False
    if len(scores_data[i]) < 2:
        print(f"Invalid scores data for frame {i}")
        valid_frames = False

if valid_frames:
    print("All frames have valid tracking and scores data.")
else:
    print("Some frames have missing or invalid data. Consider interpolation.")


All frames have valid tracking and scores data.


In [39]:
!pip3 install ffmpeg-python

  Using cached ffmpeg_python-0.2.0-py3-none-any.whl.metadata (1.7 kB)
Using cached ffmpeg_python-0.2.0-py3-none-any.whl (25 kB)


In [35]:
!pip uninstall -y ffmpeg

 

Found existing installation: ffmpeg 1.4
Uninstalling ffmpeg-1.4:
  Successfully uninstalled ffmpeg-1.4


In [48]:
!pip install ffmpeg-python


In [47]:
#frame
#bounding boxes
# x1, y1, x2, y2 coordinates of face
#They are represented as [x1, y1, x2, y2],
#where (x1, y1) is the top-left corner and (x2, y2) is the bottom-right corner of the bounding box.
#predictions


In [24]:
 
import ffmpeg
import pickle
import numpy as np

In [25]:
 !pip install ffmpeg-python


In [60]:
!sudo apt-get install ffmpeg


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 68 not upgraded.


In [33]:
 
!ls /kaggle/working/TalkNet-ASD/demo/trimmed_video


ls: cannot access '/content/TalkNet-ASD/demo/trimmed_video/pyavi/': No such file or directory


In [62]:
#Step 1,So here using ffmpeg or moviepy you will extract audio from original file.


import subprocess

def extract_audio(video_path, output_audio_path):
    try:
        command = [
            'ffmpeg',
            '-i', video_path,
            '-vn',  # No video
            '-acodec', 'mp3',
            '-ab', '192k',
            output_audio_path
        ]
        subprocess.run(command, check=True)
    except subprocess.CalledProcessError as e:
        print('Error extracting audio:', str(e))


        
video_path="/kaggle/working/TalkNet-ASD/demo/upd_trimmed_video/pyavi/video_out.avi"
output_audio_path="/kaggle/working/TalkNet-ASD/demo/final_output_with_audio.mp4"
extract_audio(video_path, output_audio_path)


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [78]:
##sample  code that is giving same video on topa and bottom

def interpolate_scores_data(scores_data, num_frames):
    interpolated_scores = []

    for frame_index in range(num_frames):
        if frame_index < len(scores_data) and isinstance(scores_data[frame_index], (list, np.ndarray)) and len(scores_data[frame_index]) == 2:
            interpolated_scores.append(scores_data[frame_index])
        else:
            # Find the nearest frames with valid data for interpolation
            prev_index = frame_index - 1
            next_index = frame_index + 1
            while prev_index >= 0 and (prev_index >= len(scores_data) or not isinstance(scores_data[prev_index], (list, np.ndarray)) or len(scores_data[prev_index]) != 2):
                prev_index -= 1
            while next_index < len(scores_data) and (not isinstance(scores_data[next_index], (list, np.ndarray)) or len(scores_data[next_index]) != 2):
                next_index += 1

            if prev_index >= 0 and next_index < len(scores_data):
                # Interpolate between previous and next valid scores
                prev_scores = np.array(scores_data[prev_index])
                next_scores = np.array(scores_data[next_index])
                alpha = (frame_index - prev_index) / (next_index - prev_index)
                interpolated_scores.append((1 - alpha) * prev_scores + alpha * next_scores)
            elif prev_index >= 0:
                # Extrapolate using the previous valid frame's scores
                interpolated_scores.append(scores_data[prev_index])
            elif next_index < len(scores_data):
                # Extrapolate using the next valid frame's scores
                interpolated_scores.append(scores_data[next_index])
            else:
                # If no valid data is available, use default scores [0, 0]
                interpolated_scores.append([0, 0])

    return interpolated_scores

# Updated function to reframe video
def reframe_video(input_video_path, output_video_path, tracks_data, faces_data, scores_data):
    # Load the video
    cap = cv2.VideoCapture(input_video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # Interpolate missing tracking and scores data
    tracks_data = interpolate_tracking_data(tracks_data, num_frames)
    scores_data = interpolate_scores_data(scores_data, num_frames)
    
    # Set the new video dimensions for a vertical screen
    new_width = 720
    new_height = 1280
    
    # Set up the output writer
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (new_width, new_height))
    
    frame_index = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Extract coordinates for the two most important people
        if frame_index >= len(tracks_data) or frame_index >= len(scores_data):
            print(f"Skipping frame {frame_index} due to insufficient tracking or score data.")
            frame_index += 1
            continue

        person1_coords = tracks_data[frame_index][0]
        person2_coords = tracks_data[frame_index][1]

        # Determine who should be on the top and who on the bottom based on scores
        if scores_data[frame_index][0] > scores_data[frame_index][1]:
            top_person_coords = person1_coords
            bottom_person_coords = person2_coords
        else:
            top_person_coords = person2_coords
            bottom_person_coords = person1_coords

        # Ensure coordinates are valid and convert them to integers
        if len(top_person_coords) < 4 or len(bottom_person_coords) < 4:
            print(f"Invalid coordinates for frame {frame_index}")
            frame_index += 1
            continue

        top_person_coords = [int(coord) for coord in top_person_coords]
        bottom_person_coords = [int(coord) for coord in bottom_person_coords]

        # Extract regions for each person
        top_person_frame = frame[
            max(0, top_person_coords[1]):min(height, top_person_coords[3]),
            max(0, top_person_coords[0]):min(width, top_person_coords[2])
        ]

        bottom_person_frame = frame[
            max(0, bottom_person_coords[1]):min(height, bottom_person_coords[3]),
            max(0, bottom_person_coords[0]):min(width, bottom_person_coords[2])
        ]

        # Resize each person's frame to the new dimensions
        top_person_frame_resized = cv2.resize(top_person_frame, (new_width, new_height // 2))
        bottom_person_frame_resized = cv2.resize(bottom_person_frame, (new_width, new_height // 2))

        # Combine the frames vertically
        combined_frame = np.vstack((top_person_frame_resized, bottom_person_frame_resized))

        # Write the combined frame to the output video
        out.write(combined_frame)
        
        frame_index += 1

    # Release everything
    cap.release()
    out.release()




input_video_path = '/kaggle/working/TalkNet-ASD/demo/upd_trimmed_video/pyavi/video_out.avi'
output_audio_path = '/kaggle/working/TalkNet-ASD/demo/extracted_audio.mp3'
output_video_path = '/kaggle/working/TalkNet-ASD/demo/ssss_reframed_video.avi'
final_output_path = '/kaggle/working/TalkNet-ASD/demo/final_output_with_audio.mp4'
reframe_video(input_video_path, output_video_path, tracks_data, faces_data, scores_data)

KeyboardInterrupt: 

In [90]:
 import pickle

# Define the paths to the pickle files
tracks_pkl_path = "/kaggle/working/TalkNet-ASD/demo/upd_trimmed_video/pywork/tracks.pckl"

# Load the tracks pickle file
with open(tracks_pkl_path, 'rb') as f:
    tracks_data = pickle.load(f)

# Inspect the structure
print(f"Total number of frames in tracks_data: {len(tracks_data)}")

# Print the first 5 frames
#for i in range(min(5, len(tracks_data))):
   # print(f"Frame {i}: {tracks_data[i]}")


Total number of frames in tracks_data: 12


In [91]:
scores_pkl_path = "/kaggle/working/TalkNet-ASD/demo/upd_trimmed_video/pywork/scores.pckl"

# Load the scores pickle file
with open(scores_pkl_path, 'rb') as f:
    scores_data = pickle.load(f)

# Inspect the structure
print(f"Total number of frames in scores_data: {len(scores_data)}")

# Print the first 5 frames
#for i in range(min(5, len(scores_data))):
    #print(f"Frame {i}: {scores_data[i]}")


Total number of frames in scores_data: 12


In [93]:
len(tracks_data)

12

In [94]:
len(scores_data)

12

In [109]:
'''import cv2
import numpy as np
import pickle
import subprocess
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d

# ---------------------------
# Step 1: Load Pickle Files
# ---------------------------

def load_pickle_file(path):
    """
    Loads a pickle file from the specified path.
    
    Args:
        path (str): Path to the pickle file.
    
    Returns:
        The loaded pickle data.
    """
    with open(path, 'rb') as f:
        return pickle.load(f)

# Define the paths to the pickle files
tracks_pkl_path = "/kaggle/working/TalkNet-ASD/demo/upd_trimmed_video/pywork/tracks.pckl"
scores_pkl_path = "/kaggle/working/TalkNet-ASD/demo/upd_trimmed_video/pywork/scores.pckl"

# Load the pickle files
tracks_data = load_pickle_file(tracks_pkl_path)
scores_data = load_pickle_file(scores_pkl_path)

# ---------------------------
# Step 2: Verify Data Structures
# ---------------------------

def verify_data_structures(tracks_data, scores_data):
    """
    Verifies and prints the structure of tracks_data and scores_data.
    
    Args:
        tracks_data (list): List of track dictionaries.
        scores_data (list): List or array of scores.
    """
    num_frames_tracks = len(tracks_data)
    num_frames_scores = len(scores_data)
    print(f"Number of frames in tracks_data: {num_frames_tracks}")
    print(f"Number of frames in scores_data: {num_frames_scores}")

    if num_frames_tracks != num_frames_scores:
        print("Warning: Number of frames in tracks_data and scores_data do not match.")

    # Check the first 3 frames
    for i in range(min(3, num_frames_tracks, num_frames_scores)):
        print(f"\nFrame {i}:")
        print(f"  Tracks: {tracks_data[i]}")
        print(f"  Scores: {scores_data[i]}")
        print(f"  Type of tracks_data[{i}]: {type(tracks_data[i])}")
        print(f"  Number of tracks in frame {i}: {len(tracks_data[i]['track']['bbox']) if 'track' in tracks_data[i] else 0}")
        print(f"  Type of scores_data[{i}]: {type(scores_data[i])}")
        print(f"  Number of scores in frame {i}: {len(scores_data[i]) if isinstance(scores_data[i], (list, np.ndarray)) else 0}")

verify_data_structures(tracks_data, scores_data)

# ---------------------------
# Step 3: Define Interpolation Functions
# ---------------------------

def extract_bounding_boxes(tracks_data, num_frames):
    """
    Extracts bounding boxes from tracks_data for each frame.
    
    Args:
        tracks_data (list): List of track dictionaries.
        num_frames (int): Total number of frames in the video.
    
    Returns:
        List[List[List[int]]]: A list where each element is a list of bounding boxes for that frame.
    """
    bounding_boxes_per_frame = [[] for _ in range(num_frames)]

    for track_idx, track in enumerate(tracks_data):
        if 'track' not in track or 'bbox' not in track['track']:
            continue  # Skip if 'track' or 'bbox' is missing

        frames = track['track']['frame']
        bboxes = track['track']['bbox']

        for frame_idx, bbox in zip(frames, bboxes):
            if frame_idx < num_frames:
                # Ensure bbox has exactly 4 elements
                if isinstance(bbox, (list, np.ndarray)) and len(bbox) == 4:
                    bounding_boxes_per_frame[frame_idx].append(bbox.tolist())
                else:
                    print(f"Warning: Invalid bbox format in track {track_idx} at frame {frame_idx}: {bbox}")
                    bounding_boxes_per_frame[frame_idx].append([0, 0, 0, 0])  # Placeholder for invalid bbox

    return bounding_boxes_per_frame

def interpolate_missing_data(data_per_frame, num_frames):
    """
    Interpolates missing data for frames.
    
    Args:
        data_per_frame (List[List]): List of data (e.g., bounding boxes or scores) per frame.
        num_frames (int): Total number of frames in the video.
    
    Returns:
        List: Data per frame with missing values interpolated.
    """
    interpolated_data = [None] * num_frames

    valid_indices = [i for i, data in enumerate(data_per_frame) if data]
    if len(valid_indices) < 2:
        # If not enough valid data, fill with default values
        for i in range(num_frames):
            interpolated_data[i] = data_per_frame[valid_indices[0]] if valid_indices else []
        return interpolated_data

    for i in valid_indices:
        interpolated_data[i] = data_per_frame[i]

    # Interpolate missing frames
    for i in range(1, len(valid_indices)):
        start_idx = valid_indices[i - 1]
        end_idx = valid_indices[i]
        start_data = np.array(data_per_frame[start_idx])
        end_data = np.array(data_per_frame[end_idx])
        for j in range(start_idx + 1, end_idx):
            alpha = (j - start_idx) / (end_idx - start_idx)
            interpolated_data[j] = ((1 - alpha) * start_data + alpha * end_data).tolist()

    return interpolated_data

def extract_scores_per_frame(tracks_data, scores_data, num_frames):
    """
    Extracts and aligns scores with bounding boxes for each frame.
    
    Args:
        tracks_data (list): List of track dictionaries.
        scores_data (list): List of score arrays.
        num_frames (int): Total number of frames in the video.
    
    Returns:
        List[List[float]]: A list where each element is a list of scores for the bounding boxes in that frame.
    """
    scores_per_frame = [[] for _ in range(num_frames)]
    num_tracks = len(tracks_data)

    for track_idx, track in enumerate(tracks_data):
        if 'proc_track' not in track or 's' not in track['proc_track']:
            continue  # Skip if 'proc_track' or 's' is missing

        track_scores = track['proc_track']['s']
        frames = track['track']['frame']

        # Ensure track_scores is a list or numpy array
        if not isinstance(track_scores, (list, np.ndarray)):
            print(f"Warning: Invalid scores format in track {track_idx}: {track_scores}")
            continue

        for frame_idx, score in zip(frames, track_scores):
            if frame_idx < num_frames:
                # Handle cases where scores_data has more scores than tracks
                if isinstance(score, (int, float, np.float32, np.float64)):
                    scores_per_frame[frame_idx].append(float(score))
                else:
                    print(f"Warning: Invalid score type in track {track_idx} at frame {frame_idx}: {score}")
                    scores_per_frame[frame_idx].append(0.0)  # Default score for invalid entries

    return scores_per_frame

# Extract bounding boxes per frame
num_frames = len(tracks_data)
bounding_boxes_per_frame = extract_bounding_boxes(tracks_data, num_frames)

# Interpolate missing bounding boxes
bounding_boxes_per_frame = interpolate_missing_data(bounding_boxes_per_frame, num_frames)

# Extract and align scores per frame
aligned_scores = extract_scores_per_frame(tracks_data, scores_data, num_frames)

# Interpolate missing scores
aligned_scores = interpolate_missing_data(aligned_scores, num_frames)

# ---------------------------
# Step 4: Define the Reframe Function
# ---------------------------

def reframe_video(input_video_path, output_video_path, bounding_boxes_per_frame, aligned_scores):
    """
    Reframes the video by splitting it vertically into top and bottom sections,
    each focusing on different individuals based on their scores.
    
    Args:
        input_video_path (str): Path to the input video file.
        output_video_path (str): Path to save the reframed output video.
        bounding_boxes_per_frame (List[List[List[int]]]): Bounding boxes per frame.
        aligned_scores (List[List[float]]): Scores aligned with bounding boxes per frame.
    """
    # Load the video
    cap = cv2.VideoCapture(input_video_path)
    if not cap.isOpened():
        print(f"Error: Cannot open video file {input_video_path}")
        return

    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    print(f"Video FPS: {fps}, Width: {width}, Height: {height}, Total Frames: {num_frames}")

    # Set the new video dimensions for a vertical screen
    new_width = 720
    new_height = 1280

    # Set up the output writer
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (new_width, new_height))

    frame_index = 0
    last_bottom_person_frame = None

    while frame_index < num_frames:
        ret, frame = cap.read()
        if not ret:
            print(f"End of video reached at frame {frame_index}.")
            break

        # Debug information
        print(f"Processing frame {frame_index}")

        # Get bounding boxes and scores for the current frame
        current_bboxes = bounding_boxes_per_frame[frame_index] if frame_index < len(bounding_boxes_per_frame) else []
        current_scores = aligned_scores[frame_index] if frame_index < len(aligned_scores) else []

        if not current_bboxes or len(current_bboxes) < 2 or not current_scores or len(current_scores) < 2:
            print(f"Warning: Not enough bounding boxes or scores data for frame {frame_index}. Using placeholder frames.")
            # Create blank frames
            top_person_frame = np.zeros((new_height // 2, new_width, 3), dtype=np.uint8)
            bottom_person_frame = last_bottom_person_frame if last_bottom_person_frame is not None else np.zeros((new_height // 2, new_width, 3), dtype=np.uint8)
            combined_frame = np.vstack

# ---------------------------
# Step 5: Execute the Reframing Process
# ---------------------------

input_video_path = '/kaggle/working/TalkNet-ASD/demo/upd_trimmed_video/pyavi/video_out.avi'
output_audio_path = '/kaggle/working/TalkNet-ASD/demo/extracted_audio.mp3'
output_video_path = '/kaggle/working/TalkNet-ASD/demo/ssss_reframed_video.avi'
final_output_path = '/kaggle/working/TalkNet-ASD/demo/final_output_with_audio.mp4'

# Now reframe the video
reframe_video(input_video_path, output_video_path, bounding_boxes_per_frame, aligned_scores)

# ---------------------------
# Step 6: (Optional) Combine Audio with Reframed Video
# ---------------------------

def combine_audio_video(reframed_video_path, audio_path, final_output_path):
    """
    Combines audio with the reframed video using FFmpeg.
    
    Args:
        reframed_video_path (str): Path to the reframed video file.
        audio_path (str): Path to the extracted audio file.
        final_output_path (str): Path to save the final video with audio.
    """
    command = [
        'ffmpeg',
        '-y',  # Overwrite output file if it exists
        '-i', reframed_video_path,
        '-i', audio_path,
        '-c:v', 'copy',
        '-c:a', 'aac',
        final_output_path
    ]
    try:
        subprocess.run(command, check=True)
        print(f"Final video with audio saved to {final_output_path}")
    except subprocess.CalledProcessError as e:
        print(f"Error combining audio and video: {e}")

# Combine the audio and video
combine_audio_video(output_video_path, output_audio_path, final_output_path)'''

'import cv2\nimport numpy as np\nimport pickle\nimport subprocess\nimport matplotlib.pyplot as plt\nfrom scipy.interpolate import interp1d\n\n# ---------------------------\n# Step 1: Load Pickle Files\n# ---------------------------\n\ndef load_pickle_file(path):\n    """\n    Loads a pickle file from the specified path.\n    \n    Args:\n        path (str): Path to the pickle file.\n    \n    Returns:\n        The loaded pickle data.\n    """\n    with open(path, \'rb\') as f:\n        return pickle.load(f)\n\n# Define the paths to the pickle files\ntracks_pkl_path = "/kaggle/working/TalkNet-ASD/demo/upd_trimmed_video/pywork/tracks.pckl"\nscores_pkl_path = "/kaggle/working/TalkNet-ASD/demo/upd_trimmed_video/pywork/scores.pckl"\n\n# Load the pickle files\ntracks_data = load_pickle_file(tracks_pkl_path)\nscores_data = load_pickle_file(scores_pkl_path)\n\n# ---------------------------\n# Step 2: Verify Data Structures\n# ---------------------------\n\ndef verify_data_structures(tracks_dat

Steps to Diagnose the Issue

a. Verify the Contents of tracks_data and scores_data

Before processing, ensure that both tracks_data and scores_data contain data for frame 0.

In [110]:
def inspect_first_frame(tracks_data, scores_data):
    frame_index = 0
    if frame_index < len(tracks_data):
        print(f"Tracks Data for Frame {frame_index}: {tracks_data[frame_index]}")
    else:
        print(f"No tracks data available for frame {frame_index}")
    
    if frame_index < len(scores_data):
        print(f"Scores Data for Frame {frame_index}: {scores_data[frame_index]}")
    else:
        print(f"No scores data available for frame {frame_index}")

inspect_first_frame(tracks_data, scores_data)


Tracks Data for Frame 0: {'track': {'frame': array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]), 'bbox': array([[301.80917358,  48.13118744, 385.58911133, 169.50073242],
       [301.80917358,  48.13118744, 385.58911133, 169.50073242],
       [301.80917358,  48.13118744, 385.58911133, 169.50073242],
       [301.80917358,  48.13118744, 385.58911133, 169.50073242],
       [301.80917358,  48.13118744, 385.58911133, 169.50073242],
       [301.80917358,  48.13118744, 385.58911133, 169.50073242],
       [301.80917358,  48.13118744, 385.58911133, 169.50073242],
       [301.80917358,  48.13118744, 385.58911133

In [111]:
print(f"Total frames in video: {num_frames}")
print(f"Total frames in tracks_data: {len(tracks_data)}")
print(f"Total frames in scores_data: {len(scores_data)}")


Total frames in video: 12
Total frames in tracks_data: 12
Total frames in scores_data: 12


In [118]:
def inspect_frame_data(tracks_data, scores_data, frame_index=13):
    print(f"\n--- Inspecting Frame {frame_index} ---")
    
    # Inspect tracks_data for frame 13
    if frame_index < len(tracks_data):
        track = tracks_data[frame_index]
        if 'track' in track and 'bbox' in track['track']:
            bboxes = track['track']['bbox']
            frames = track['track']['frame']
            print(f"Bounding Boxes: {bboxes}")
            print(f"Frames: {frames}")
        else:
            print("No 'track' or 'bbox' in tracks_data for this frame.")
    else:
        print("No tracks_data available for this frame.")

    # Inspect scores_data for frame 0
    if frame_index < len(scores_data):
        scores = scores_data[frame_index]
        print(f"Scores: {scores}")
    else:
        print("No scores_data available for this frame.")

# Call the inspection function
inspect_frame_data(tracks_data, scores_data, frame_index=11)



--- Inspecting Frame 11 ---
Bounding Boxes: [[454.41870117  72.32515717 558.50439453 216.02212524]
 [454.08630371  72.13864899 558.14221191 216.27853394]
 [454.17437744  72.23003387 557.91351318 217.26853943]
 [454.61694336  72.23212433 557.55566406 218.8143158 ]
 [455.40008545  72.75363922 558.4666748  218.20295715]
 [456.01281738  73.78868866 561.23791504 217.68803406]
 [455.86853027  74.23610687 562.17944336 217.38838196]
 [456.17871094  74.37711334 562.6885376  217.7301178 ]
 [456.80007935  74.87384796 562.86358643 217.95401001]
 [456.98773193  75.59116364 562.88342285 217.99874878]
 [456.46609497  75.63174438 562.57684326 218.25881958]
 [456.12594604  75.41072845 562.17932129 217.71652222]
 [456.12643433  74.85074615 561.88238525 217.55886841]
 [455.67974854  74.43078613 561.56408691 218.05041504]
 [455.78918457  73.71199799 562.05328369 217.57196045]
 [455.58718872  73.65837097 561.24462891 217.18745422]
 [455.59567261  73.75154114 560.81762695 216.56100464]
 [455.24337769  73.9

In [123]:
def check_frame_indices(tracks_data, frame_index=0):
    if frame_index < len(tracks_data):
        track = tracks_data[frame_index]
        if 'track' in track and 'frame' in track['track']:
            frame_numbers = track['track']['frame']
            print(f"Frame numbers in track {frame_index}: {frame_numbers}")
        else:
            print(f"No 'frame' information in tracks_data for frame {frame_index}.")
    else:
        print(f"No tracks_data available for frame {frame_index}.")

# Check frame indices for the first few tracks
for i in range(min(12, len(tracks_data))):
    check_frame_indices(tracks_data, frame_index=i)


Frame numbers in track 0: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95
 96 97 98 99]
Frame numbers in track 1: [100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117
 118 119 120 121 122 123]
Frame numbers in track 2: [ 124  125  126 ... 1691 1692 1693]
Frame numbers in track 3: [ 124  125  126 ... 1691 1692 1693]
Frame numbers in track 4: [1694 1695 1696 1697 1698 1699 1700 1701 1702 1703 1704 1705 1706 1707
 1708 1709 1710 1711 1712 1713 1714 1715 1716 1717]
Frame numbers in track 5: [1694 1695 1696 1697 1698 1699 1700 1701 1702 1703 1704 1705 1706 1707
 1708 1709 1710 1711 1712 1713 1714 1715 1716 1717]
Frame numbers in track 6: [1718 1719 1720 1721 1722 1723 1724 1725 1726 1727 1728 1729 1730 1731
 1732 1733 1734 1735 173

In [125]:
def extract_bounding_boxes(tracks_data, num_frames, frame_offset=0):
    """
    Extracts bounding boxes from tracks_data for each frame.
    
    Args:
        tracks_data (list): List of track dictionaries.
        num_frames (int): Total number of frames in the video.
        frame_offset (int): Offset to apply to frame indices (e.g., 1 if tracking starts at frame 1).
    
    Returns:
        List[List[List[int]]]: A list where each element is a list of bounding boxes for that frame.
    """
    bounding_boxes_per_frame = [[] for _ in range(num_frames)]

    for track_idx, track in enumerate(tracks_data):
        if 'track' not in track or 'bbox' not in track['track']:
            continue  # Skip if 'track' or 'bbox' is missing

        frames = track['track']['frame']
        bboxes = track['track']['bbox']

        for frame_idx, bbox in zip(frames, bboxes):
            adjusted_frame_idx = frame_idx - frame_offset
            if 0 <= adjusted_frame_idx < num_frames:
                if isinstance(bbox, (list, np.ndarray)) and len(bbox) == 4:
                    bounding_boxes_per_frame[adjusted_frame_idx].append(bbox.tolist())
                else:
                    print(f"Warning: Invalid bbox format in track {track_idx} at frame {frame_idx}: {bbox}")
                    bounding_boxes_per_frame[adjusted_frame_idx].append([0, 0, 0, 0])  # Placeholder for invalid bbox

    return bounding_boxes_per_frame

extract_bounding_boxes(tracks_data, num_frames, frame_offset=0)


[[[301.8091735839844, 48.131187438964844, 385.589111328125, 169.500732421875]],
 [[301.8091735839844, 48.131187438964844, 385.589111328125, 169.500732421875]],
 [[301.8091735839844, 48.131187438964844, 385.589111328125, 169.500732421875]],
 [[301.8091735839844, 48.131187438964844, 385.589111328125, 169.500732421875]],
 [[301.8091735839844, 48.131187438964844, 385.589111328125, 169.500732421875]],
 [[301.8091735839844, 48.131187438964844, 385.589111328125, 169.500732421875]],
 [[301.8091735839844, 48.131187438964844, 385.589111328125, 169.500732421875]],
 [[301.8091735839844, 48.131187438964844, 385.589111328125, 169.500732421875]],
 [[301.8091735839844, 48.131187438964844, 385.589111328125, 169.500732421875]],
 [[301.8091735839844, 48.131187438964844, 385.589111328125, 169.500732421875]],
 [[301.8091735839844, 48.131187438964844, 385.589111328125, 169.500732421875]],
 [[301.8091735839844, 48.131187438964844, 385.589111328125, 169.500732421875]]]

In [126]:
print("\n=== Verification After Extraction and Interpolation ===")
print(f"Bounding Boxes for Frame 0: {bounding_boxes_per_frame[0]}")
print(f"Scores for Frame 0: {aligned_scores[0]}")



=== Verification After Extraction and Interpolation ===
Bounding Boxes for Frame 0: [[301.8091735839844, 48.131187438964844, 385.589111328125, 169.500732421875]]
Scores for Frame 0: [60.68477249145508]


In [132]:
pip install moviepy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.3/388.3 kB 5.3 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for moviepy: filename=moviepy-1.0.3-py3-none-any.whl size=110721 sha256=ab778373d4f7209ff277ecff6e67daf7ad244829ce6aa2c961452cd644112cf5
  Stored in directory: /root/.cache/pip/wheels/96/32/2d/e10123bd88fbfc02fed53cc18c80a171d3c87479ed845fa7c1
Successfully built moviepy
  Attempting uninstall: decorator
    Found existing installation: decorator 5.1.1
    Uninstalling decorator-5.1.1:
      Successfully uninstalled decorator-5.1.1
Note: you may need to restart the kernel to use updated packages.


In [141]:
import cv2
import numpy as np
import pickle
import subprocess
import os
from moviepy.editor import VideoFileClip, AudioFileClip
from scipy.interpolate import interp1d

# ---------------------------
# Step 1: Load Pickle Files
# ---------------------------

def load_pickle_file(path):
    """
    Loads a pickle file from the specified path.

    Args:
        path (str): Path to the pickle file.

    Returns:
        The loaded pickle data.
    """
    with open(path, 'rb') as f:
        return pickle.load(f)

# Define the paths to the pickle files
# Replace these paths with the actual paths to your pickle files
tracks_pkl_path = "/kaggle/working/TalkNet-ASD/demo/upd_trimmed_video/pywork/tracks.pckl"
scores_pkl_path = "/kaggle/working/TalkNet-ASD/demo/upd_trimmed_video/pywork/scores.pckl"

# Load the pickle files
tracks_data = load_pickle_file(tracks_pkl_path)
scores_data = load_pickle_file(scores_pkl_path)

# ---------------------------
# Step 2: Verify Data Structures
# ---------------------------

def verify_data_structures(tracks_data, scores_data, num_frames):
    """
    Verifies and prints the structure of tracks_data and scores_data.

    Args:
        tracks_data (list): List of track dictionaries.
        scores_data (list): List or array of scores.
        num_frames (int): Total number of frames in the video.
    """
    print(f"Number of tracks: {len(tracks_data)}")
    print(f"Number of frames: {num_frames}")

    # Check the first few tracks
    for track_idx, track in enumerate(tracks_data[:3]):
        print(f"\nTrack {track_idx}:")
        print(f"  Frames: {track['track']['frame']}")
        print(f"  Bounding Boxes: {track['track']['bbox']}")
        print(f"  Processed Track X: {track['proc_track']['x']}")
        print(f"  Processed Track Y: {track['proc_track']['y']}")
        print(f"  Processed Track Scores: {track['proc_track']['s']}")

    # Check the first few frames
    for frame_idx in range(min(3, num_frames)):
        print(f"\nFrame {frame_idx}:")
        print(f"  Scores: {scores_data[frame_idx]}")
        # You can add more details as needed

# Optional: Uncomment to verify data structures
# You might need to adjust `num_frames` based on your video
# num_frames = 100  # Example value; replace with actual frame count
# verify_data_structures(tracks_data, scores_data, num_frames)

# ---------------------------
# Step 3: Define Helper Functions
# ---------------------------

def extract_bounding_boxes(tracks_data, num_frames):
    """
    Extracts bounding boxes from tracks_data for each frame.

    Args:
        tracks_data (list): List of track dictionaries.
        num_frames (int): Total number of frames in the video.

    Returns:
        List[List[List[float]]]: A list where each element is a list of bounding boxes for that frame.
    """
    bounding_boxes_per_frame = [[] for _ in range(num_frames)]

    for track_idx, track in enumerate(tracks_data):
        if 'track' not in track or 'bbox' not in track['track']:
            continue  # Skip if 'track' or 'bbox' is missing

        frames = track['track']['frame']
        bboxes = track['track']['bbox']

        for frame_num, bbox in zip(frames, bboxes):
            if 0 <= frame_num < num_frames:
                # Ensure bbox has exactly 4 elements
                if isinstance(bbox, (list, np.ndarray)) and len(bbox) == 4:
                    bounding_boxes_per_frame[frame_num].append(bbox.tolist())
                else:
                    print(f"Warning: Invalid bbox format in track {track_idx} at frame {frame_num}: {bbox}")
                    bounding_boxes_per_frame[frame_num].append([0, 0, 0, 0])  # Placeholder for invalid bbox

    return bounding_boxes_per_frame

def extract_scores_per_frame(tracks_data, scores_data, num_frames):
    """
    Extracts and aligns scores with bounding boxes for each frame.

    Args:
        tracks_data (list): List of track dictionaries.
        scores_data (list): List of score arrays.
        num_frames (int): Total number of frames in the video.

    Returns:
        List[List[float]]: A list where each element is a list of scores for the bounding boxes in that frame.
    """
    scores_per_frame = [[] for _ in range(num_frames)]

    for track_idx, track in enumerate(tracks_data):
        if 'proc_track' not in track or 's' not in track['proc_track']:
            continue  # Skip if 'proc_track' or 's' is missing

        track_scores = track['proc_track']['s']
        frames = track['track']['frame']

        # Ensure track_scores is a list or numpy array
        if not isinstance(track_scores, (list, np.ndarray)):
            print(f"Warning: Invalid scores format in track {track_idx}: {track_scores}")
            continue

        for frame_num, score in zip(frames, track_scores):
            if 0 <= frame_num < num_frames:
                # Handle cases where scores_data has more scores than tracks
                if isinstance(score, (int, float, np.float32, np.float64)):
                    scores_per_frame[frame_num].append(float(score))
                else:
                    print(f"Warning: Invalid score type in track {track_idx} at frame {frame_num}: {score}")
                    scores_per_frame[frame_num].append(0.0)  # Default score for invalid entries

    return scores_per_frame

def interpolate_missing_data(data_per_frame, num_frames):
    """
    Interpolates missing data for frames.

    Args:
        data_per_frame (List[List]): List of data (e.g., bounding boxes or scores) per frame.
        num_frames (int): Total number of frames in the video.

    Returns:
        List: Data per frame with missing values interpolated.
    """
    interpolated_data = [None] * num_frames

    # Find frames with valid data
    valid_indices = [i for i, data in enumerate(data_per_frame) if data]
    if len(valid_indices) < 2:
        # If not enough valid data, fill with default values
        for i in range(num_frames):
            interpolated_data[i] = data_per_frame[valid_indices[0]] if valid_indices else []
        return interpolated_data

    for i in valid_indices:
        interpolated_data[i] = data_per_frame[i]

    # Interpolate missing frames
    for i in range(1, len(valid_indices)):
        start_idx = valid_indices[i - 1]
        end_idx = valid_indices[i]
        start_data = np.array(data_per_frame[start_idx])
        end_data = np.array(data_per_frame[end_idx])
        for j in range(start_idx + 1, end_idx):
            alpha = (j - start_idx) / (end_idx - start_idx)
            interpolated_data[j] = ((1 - alpha) * start_data + alpha * end_data).tolist()

    # Fill any remaining None with empty lists
    for i in range(num_frames):
        if interpolated_data[i] is None:
            interpolated_data[i] = []

    return interpolated_data

# ---------------------------
# Step 4: Reframe Video Function
# ---------------------------

def reframe_video(
    input_video_path,
    output_video_path,
    bounding_boxes_per_frame,
    scores_per_frame,
    aspect_ratio=(9, 16)
):
    """
    Reframes the video by placing the most active person at the top and another person at the bottom
    on a vertical screen with the specified aspect ratio.

    Args:
        input_video_path (str): Path to the input video file.
        output_video_path (str): Path to save the reframed output video.
        bounding_boxes_per_frame (List[List[List[float]]]): Bounding boxes per frame.
        scores_per_frame (List[List[float]]): Scores aligned with bounding boxes per frame.
        aspect_ratio (tuple): Desired aspect ratio (width, height). Default is (9, 16).
    """
    # Load the video
    cap = cv2.VideoCapture(input_video_path)

    if not cap.isOpened():
        print(f"Error: Cannot open video file {input_video_path}")
        return

    fps = cap.get(cv2.CAP_PROP_FPS)
    original_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    original_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    print(f"Video FPS: {fps}, Original Width: {original_width}, Original Height: {original_height}, Total Frames: {num_frames}")

    # Define new video dimensions based on aspect ratio (e.g., 720x1280 for 9:16)
    new_width = 720
    new_height = 1280

    # Calculate frame size for top and bottom sections
    section_height = new_height // 2
    section_width = new_width

    # Set up the output writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Using mp4v for MP4 container
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (new_width, new_height))

    frame_index = 0
    last_bottom_person_frame = np.zeros((section_height, section_width, 3), dtype=np.uint8)

    while frame_index < num_frames:
        ret, frame = cap.read()
        if not ret:
            print(f"End of video reached at frame {frame_index}.")
            break

        # Debug information
        print(f"Processing frame {frame_index + 1}/{num_frames}")

        # Get bounding boxes and scores for the current frame
        current_bboxes = bounding_boxes_per_frame[frame_index] if frame_index < len(bounding_boxes_per_frame) else []
        current_scores = scores_per_frame[frame_index] if frame_index < len(scores_per_frame) else []

        if not current_bboxes or len(current_bboxes) < 1 or not current_scores or len(current_scores) < 1:
            print(f"Warning: Not enough bounding boxes or scores data for frame {frame_index}. Using placeholder frames.")
            # Create blank frames
            top_person_frame = np.zeros((section_height, section_width, 3), dtype=np.uint8)
            bottom_person_frame = last_bottom_person_frame.copy()
            combined_frame = np.vstack((top_person_frame, bottom_person_frame))
            out.write(combined_frame)
            frame_index += 1
            continue

        # Pair each bounding box with its corresponding score
        people = [{'bbox': bbox, 'score': score} for bbox, score in zip(current_bboxes, current_scores) if len(bbox) == 4]

        if not people:
            print(f"Warning: No valid people detected in frame {frame_index}. Using placeholder frames.")
            top_person_frame = np.zeros((section_height, section_width, 3), dtype=np.uint8)
            bottom_person_frame = last_bottom_person_frame.copy()
            combined_frame = np.vstack((top_person_frame, bottom_person_frame))
            out.write(combined_frame)
            frame_index += 1
            continue

        # Sort people based on their scores to determine top and bottom
        people_sorted = sorted(people, key=lambda x: x['score'], reverse=True)

        # Assign top and bottom persons
        top_person = people_sorted[0]
        bottom_person = people_sorted[1] if len(people_sorted) > 1 else top_person  # If only one person, duplicate

        # Debug information
        print(f"Top person score: {top_person['score']}, Bottom person score: {bottom_person['score']}")

        # Function to extract and resize frame section based on bounding box
        def extract_and_resize(bbox):
            x1, y1, x2, y2 = map(int, bbox)
            # Ensure coordinates are within frame bounds
            x1 = max(0, min(original_width - 1, x1))
            y1 = max(0, min(original_height - 1, y1))
            x2 = max(0, min(original_width - 1, x2))
            y2 = max(0, min(original_height - 1, y2))
            if y1 >= y2 or x1 >= x2:
                print(f"Invalid bbox coordinates: {bbox}. Using blank frame.")
                return np.zeros((section_height, section_width, 3), dtype=np.uint8)
            cropped = frame[y1:y2, x1:x2]
            # Handle cases where cropped area is too small
            if cropped.size == 0:
                print(f"Cropped frame size is zero for bbox: {bbox}. Using blank frame.")
                return np.zeros((section_height, section_width, 3), dtype=np.uint8)
            try:
                resized = cv2.resize(cropped, (section_width, section_height))
            except Exception as e:
                print(f"Error resizing cropped frame: {e}. Using blank frame.")
                resized = np.zeros((section_height, section_width, 3), dtype=np.uint8)
            return resized

        # Extract and resize top and bottom person frames
        top_person_frame = extract_and_resize(top_person['bbox'])
        bottom_person_frame = extract_and_resize(bottom_person['bbox'])

        # Update last known bottom person frame if a valid frame is extracted
        if not np.array_equal(bottom_person_frame, np.zeros((section_height, section_width, 3), dtype=np.uint8)):
            last_bottom_person_frame = bottom_person_frame.copy()

        # Combine the two frames vertically
        combined_frame = np.vstack((top_person_frame, last_bottom_person_frame))

        # Write the combined frame to the output video
        out.write(combined_frame)

        frame_index += 1

    # Release resources
    cap.release()
    out.release()
    print(f"Reframed video saved to {output_video_path}")


# ---------------------------
# Step 7: Main Execution
# ---------------------------

def main():
    # Define paths
    input_video_path = '/kaggle/working/TalkNet-ASD/demo/upd_trimmed_video/pyavi/video_out.avi'
    extracted_audio_path = '/kaggle/working/TalkNet-ASD/demo/extracted_audio.mp3'
    reframed_video_path = '/kaggle/working/TalkNet-ASD/demo/sxsxs_reframed_video.avi'
    final_output_path = '/kaggle/working/TalkNet-ASD/demo/final_output_with_audio.mp4'

    # Check if input video exists
    if not os.path.exists(input_video_path):
        print(f"Error: Input video file '{input_video_path}' does not exist.")
        return

    # Determine number of frames from the video
    cap = cv2.VideoCapture(input_video_path)
    if not cap.isOpened():
        print(f"Error: Cannot open video file {input_video_path}")
        return
    num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    cap.release()

    # Extract bounding boxes and scores per frame
    bounding_boxes_per_frame = extract_bounding_boxes(tracks_data, num_frames)
    scores_per_frame = extract_scores_per_frame(tracks_data, scores_data, num_frames)

    # Interpolate missing bounding boxes and scores
    bounding_boxes_per_frame = interpolate_missing_data(bounding_boxes_per_frame, num_frames)
    scores_per_frame = interpolate_missing_data(scores_per_frame, num_frames)

    

    # Step 2: Reframe video
    reframe_video(
        input_video_path,
        reframed_video_path,
        bounding_boxes_per_frame,
        scores_per_frame,
        aspect_ratio=(9, 16)
    )
 

    

if __name__ == "__main__":
    main()


Video FPS: 25.0, Original Width: 640, Original Height: 360, Total Frames: 2054
Processing frame 1/2054
Top person score: 60.68477249145508, Bottom person score: 60.68477249145508
Processing frame 2/2054
Top person score: 60.68477249145508, Bottom person score: 60.68477249145508
Processing frame 3/2054
Top person score: 60.68477249145508, Bottom person score: 60.68477249145508
Processing frame 4/2054
Top person score: 60.68477249145508, Bottom person score: 60.68477249145508
Processing frame 5/2054
Top person score: 60.68477249145508, Bottom person score: 60.68477249145508
Processing frame 6/2054
Top person score: 60.68477249145508, Bottom person score: 60.68477249145508
Processing frame 7/2054
Top person score: 60.68477249145508, Bottom person score: 60.68477249145508
Processing frame 8/2054
Top person score: 60.68477249145508, Bottom person score: 60.68477249145508
Processing frame 9/2054
Top person score: 60.68477249145508, Bottom person score: 60.68477249145508
Processing frame 10/2

In [135]:
ls /kaggle/working/TalkNet-ASD/demo/extracted_audio.mp3


/kaggle/working/TalkNet-ASD/demo/extracted_audio.mp3


In [142]:
'''import cv2
import numpy as np
import pickle
import subprocess
import os
from moviepy.editor import VideoFileClip, AudioFileClip
from scipy.interpolate import interp1d

# ---------------------------
# Step 1: Load Pickle Files
# ---------------------------

def load_pickle_file(path):
    """
    Loads a pickle file from the specified path.

    Args:
        path (str): Path to the pickle file.

    Returns:
        The loaded pickle data.
    """
    with open(path, 'rb') as f:
        return pickle.load(f)

# Define the paths to the pickle files
# Replace these paths with the actual paths to your pickle files
tracks_pkl_path = "/kaggle/working/TalkNet-ASD/demo/upd_trimmed_video/pywork/tracks.pckl"
scores_pkl_path = "/kaggle/working/TalkNet-ASD/demo/upd_trimmed_video/pywork/scores.pckl"

# Load the pickle files
tracks_data = load_pickle_file(tracks_pkl_path)
scores_data = load_pickle_file(scores_pkl_path)

# ---------------------------
# Step 2: Verify Data Structures
# ---------------------------

def verify_data_structures(tracks_data, scores_data, num_frames):
    """
    Verifies and prints the structure of tracks_data and scores_data.

    Args:
        tracks_data (list): List of track dictionaries.
        scores_data (list): List or array of scores.
        num_frames (int): Total number of frames in the video.
    """
    print(f"Number of tracks: {len(tracks_data)}")
    print(f"Number of frames: {num_frames}")

    # Check the first few tracks
    for track_idx, track in enumerate(tracks_data[:3]):
        print(f"\nTrack {track_idx}:")
        print(f"  Frames: {track['track']['frame']}")
        print(f"  Bounding Boxes: {track['track']['bbox']}")
        print(f"  Processed Track X: {track['proc_track']['x']}")
        print(f"  Processed Track Y: {track['proc_track']['y']}")
        print(f"  Processed Track Scores: {track['proc_track']['s']}")

    # Check the first few frames
    for frame_idx in range(min(3, num_frames)):
        print(f"\nFrame {frame_idx}:")
        print(f"  Scores: {scores_data[frame_idx]}")
        # You can add more details as needed

# Optional: Uncomment to verify data structures
# You might need to adjust `num_frames` based on your video
# num_frames = 100  # Example value; replace with actual frame count
# verify_data_structures(tracks_data, scores_data, num_frames)

# ---------------------------
# Step 3: Define Helper Functions
# ---------------------------

def extract_bounding_boxes(tracks_data, num_frames):
    """
    Extracts bounding boxes from tracks_data for each frame.

    Args:
        tracks_data (list): List of track dictionaries.
        num_frames (int): Total number of frames in the video.

    Returns:
        List[List[List[float]]]: A list where each element is a list of bounding boxes for that frame.
    """
    bounding_boxes_per_frame = [[] for _ in range(num_frames)]

    for track_idx, track in enumerate(tracks_data):
        if 'track' not in track or 'bbox' not in track['track']:
            continue  # Skip if 'track' or 'bbox' is missing

        frames = track['track']['frame']
        bboxes = track['track']['bbox']

        for frame_num, bbox in zip(frames, bboxes):
            if 0 <= frame_num < num_frames:
                # Ensure bbox has exactly 4 elements
                if isinstance(bbox, (list, np.ndarray)) and len(bbox) == 4:
                    bounding_boxes_per_frame[frame_num].append(bbox.tolist())
                else:
                    print(f"Warning: Invalid bbox format in track {track_idx} at frame {frame_num}: {bbox}")
                    bounding_boxes_per_frame[frame_num].append([0, 0, 0, 0])  # Placeholder for invalid bbox

    return bounding_boxes_per_frame

def extract_scores_per_frame(tracks_data, scores_data, num_frames):
    """
    Extracts and aligns scores with bounding boxes for each frame.

    Args:
        tracks_data (list): List of track dictionaries.
        scores_data (list): List of score arrays.
        num_frames (int): Total number of frames in the video.

    Returns:
        List[List[float]]: A list where each element is a list of scores for the bounding boxes in that frame.
    """
    scores_per_frame = [[] for _ in range(num_frames)]

    for track_idx, track in enumerate(tracks_data):
        if 'proc_track' not in track or 's' not in track['proc_track']:
            continue  # Skip if 'proc_track' or 's' is missing

        track_scores = track['proc_track']['s']
        frames = track['track']['frame']

        # Ensure track_scores is a list or numpy array
        if not isinstance(track_scores, (list, np.ndarray)):
            print(f"Warning: Invalid scores format in track {track_idx}: {track_scores}")
            continue

        for frame_num, score in zip(frames, track_scores):
            if 0 <= frame_num < num_frames:
                # Handle cases where scores_data has more scores than tracks
                if isinstance(score, (int, float, np.float32, np.float64)):
                    scores_per_frame[frame_num].append(float(score))
                else:
                    print(f"Warning: Invalid score type in track {track_idx} at frame {frame_num}: {score}")
                    scores_per_frame[frame_num].append(0.0)  # Default score for invalid entries

    return scores_per_frame

def interpolate_missing_data(data_per_frame, num_frames):
    """
    Interpolates missing data for frames.

    Args:
        data_per_frame (List[List]): List of data (e.g., bounding boxes or scores) per frame.
        num_frames (int): Total number of frames in the video.

    Returns:
        List: Data per frame with missing values interpolated.
    """
    interpolated_data = [None] * num_frames

    # Find frames with valid data
    valid_indices = [i for i, data in enumerate(data_per_frame) if data]
    if len(valid_indices) < 2:
        # If not enough valid data, fill with default values
        for i in range(num_frames):
            interpolated_data[i] = data_per_frame[valid_indices[0]] if valid_indices else []
        return interpolated_data

    for i in valid_indices:
        interpolated_data[i] = data_per_frame[i]

    # Interpolate missing frames
    for i in range(1, len(valid_indices)):
        start_idx = valid_indices[i - 1]
        end_idx = valid_indices[i]
        start_data = np.array(data_per_frame[start_idx])
        end_data = np.array(data_per_frame[end_idx])
        for j in range(start_idx + 1, end_idx):
            alpha = (j - start_idx) / (end_idx - start_idx)
            interpolated_data[j] = ((1 - alpha) * start_data + alpha * end_data).tolist()

    # Fill any remaining None with empty lists
    for i in range(num_frames):
        if interpolated_data[i] is None:
            interpolated_data[i] = []

    return interpolated_data

# ---------------------------
# Step 4: Reframe Video Function
# ---------------------------

def reframe_video(
    input_video_path,
    output_video_path,
    bounding_boxes_per_frame,
    scores_per_frame,
    aspect_ratio=(9, 16)
):
    """
    Reframes the video by placing the most active person at the top and another person at the bottom
    on a vertical screen with the specified aspect ratio.

    Args:
        input_video_path (str): Path to the input video file.
        output_video_path (str): Path to save the reframed output video.
        bounding_boxes_per_frame (List[List[List[float]]]): Bounding boxes per frame.
        scores_per_frame (List[List[float]]): Scores aligned with bounding boxes per frame.
        aspect_ratio (tuple): Desired aspect ratio (width, height). Default is (9, 16).
    """
    # Load the video
    cap = cv2.VideoCapture(input_video_path)

    if not cap.isOpened():
        print(f"Error: Cannot open video file {input_video_path}")
        return

    fps = cap.get(cv2.CAP_PROP_FPS)
    original_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    original_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    print(f"Video FPS: {fps}, Original Width: {original_width}, Original Height: {original_height}, Total Frames: {num_frames}")

    # Define new video dimensions based on aspect ratio (e.g., 720x1280 for 9:16)
    new_width = 720
    new_height = 1280

    # Calculate frame size for top and bottom sections
    section_height = new_height // 2
    section_width = new_width

    # Set up the output writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Using mp4v for MP4 container
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (new_width, new_height))

    frame_index = 0
    last_bottom_person_frame = np.zeros((section_height, section_width, 3), dtype=np.uint8)

    while frame_index < num_frames:
        ret, frame = cap.read()
        if not ret:
            print(f"End of video reached at frame {frame_index}.")
            break

        # Debug information
        print(f"Processing frame {frame_index + 1}/{num_frames}")

        # Get bounding boxes and scores for the current frame
        current_bboxes = bounding_boxes_per_frame[frame_index] if frame_index < len(bounding_boxes_per_frame) else []
        current_scores = scores_per_frame[frame_index] if frame_index < len(scores_per_frame) else []

        if not current_bboxes or len(current_bboxes) < 1 or not current_scores or len(current_scores) < 1:
            print(f"Warning: Not enough bounding boxes or scores data for frame {frame_index}. Using placeholder frames.")
            # Create blank frames
            top_person_frame = np.zeros((section_height, section_width, 3), dtype=np.uint8)
            bottom_person_frame = last_bottom_person_frame.copy()
            combined_frame = np.vstack((top_person_frame, bottom_person_frame))
            out.write(combined_frame)
            frame_index += 1
            continue

        # Pair each bounding box with its corresponding score
        people = [{'bbox': bbox, 'score': score} for bbox, score in zip(current_bboxes, current_scores) if len(bbox) == 4]

        if not people:
            print(f"Warning: No valid people detected in frame {frame_index}. Using placeholder frames.")
            top_person_frame = np.zeros((section_height, section_width, 3), dtype=np.uint8)
            bottom_person_frame = last_bottom_person_frame.copy()
            combined_frame = np.vstack((top_person_frame, bottom_person_frame))
            out.write(combined_frame)
            frame_index += 1
            continue

        # Sort people based on their scores to determine top and bottom
        people_sorted = sorted(people, key=lambda x: x['score'], reverse=True)

        # Assign top and bottom persons
        top_person = people_sorted[0]
        bottom_person = people_sorted[1] if len(people_sorted) > 1 else top_person  # If only one person, duplicate

        # Debug information
        print(f"Top person score: {top_person['score']}, Bottom person score: {bottom_person['score']}")

        # Function to extract and resize frame section based on bounding box
        def extract_and_resize(bbox):
            x1, y1, x2, y2 = map(int, bbox)
            # Ensure coordinates are within frame bounds
            x1 = max(0, min(original_width - 1, x1))
            y1 = max(0, min(original_height - 1, y1))
            x2 = max(0, min(original_width - 1, x2))
            y2 = max(0, min(original_height - 1, y2))
            if y1 >= y2 or x1 >= x2:
                print(f"Invalid bbox coordinates: {bbox}. Using blank frame.")
                return np.zeros((section_height, section_width, 3), dtype=np.uint8)
            cropped = frame[y1:y2, x1:x2]
            # Handle cases where cropped area is too small
            if cropped.size == 0:
                print(f"Cropped frame size is zero for bbox: {bbox}. Using blank frame.")
                return np.zeros((section_height, section_width, 3), dtype=np.uint8)
            try:
                resized = cv2.resize(cropped, (section_width, section_height))
            except Exception as e:
                print(f"Error resizing cropped frame: {e}. Using blank frame.")
                resized = np.zeros((section_height, section_width, 3), dtype=np.uint8)
            return resized

        # Extract and resize top and bottom person frames
        top_person_frame = extract_and_resize(top_person['bbox'])
        bottom_person_frame = extract_and_resize(bottom_person['bbox'])

        # Update last known bottom person frame if a valid frame is extracted
        if not np.array_equal(bottom_person_frame, np.zeros((section_height, section_width, 3), dtype=np.uint8)):
            last_bottom_person_frame = bottom_person_frame.copy()

        # Combine the two frames vertically
        combined_frame = np.vstack((top_person_frame, last_bottom_person_frame))

        # Write the combined frame to the output video
        out.write(combined_frame)

        frame_index += 1

    # Release resources
    cap.release()
    out.release()
    print(f"Reframed video saved to {output_video_path}")

# ---------------------------
# Step 5: Extract Audio from Original Video
# ---------------------------

def extract_audio(input_video_path, audio_path):
    """
    Extracts audio from the original video using FFmpeg.

    Args:
        input_video_path (str): Path to the input video file.
        output_audio_path (str): Path to save the extracted audio file.
    """
    command = [
        'ffmpeg',
        '-y',  # Overwrite output file if it exists
        '-i', input_video_path,
        '-q:a', '0',
        '-map', 'a',
          audio_path
    ]
    try:
        subprocess.run(command, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        print(f"Audio extracted to {output_audio_path}")
    except subprocess.CalledProcessError as e:
        print(f"Error extracting audio: {e.stderr.decode()}")

# ---------------------------
# Step 6: Combine Audio with Reframed Video
# ---------------------------

from moviepy.editor import VideoFileClip, AudioFileClip
import os

def combine_audio_video(reframed_video_path, audio_path, final_output_path):
    """
    Combines audio with the reframed video using MoviePy.

    Args:
        reframed_video_path (str): Path to the reframed video file.
        audio_path (str): Path to the extracted audio file.
        final_output_path (str): Path to save the final video with audio.
    """
    try:
        # Check if files exist
        if not os.path.exists(reframed_video_path):
            print(f"Error: Reframed video file '{reframed_video_path}' does not exist.")
            return

        if not os.path.exists(audio_path):
            print(f"Error: Audio file '{audio_path}' does not exist.")
            return

        # Load the video
        print("Loading reframed video...")
        video_clip = VideoFileClip(reframed_video_path)
        
        if video_clip.duration is None:
            raise ValueError(f"Video clip duration is None. Please check the video file: {reframed_video_path}")

        print(f"Video duration: {video_clip.duration} seconds")

        # Attempt to load the audio
        print("Loading audio...")
        try:
            audio_clip = AudioFileClip(audio_path)
        except Exception as e:
            print(f"Error loading audio file: {e}")
            audio_clip = None

        # Check if the audio clip loaded properly
        if audio_clip is None or audio_clip.duration is None:
            print("Warning: Audio clip could not be loaded or has no duration. Proceeding without audio.")
            final_clip = video_clip
        else:
            if audio_clip.duration <= 0:
                print("Warning: Audio clip has zero duration. Proceeding without audio.")
                final_clip = video_clip
            else:
                print(f"Audio duration: {audio_clip.duration} seconds")
                final_clip = video_clip.set_audio(audio_clip)

        # Write the final video
        print("Writing final video...")
        final_clip.write_videofile(final_output_path, codec='libx264', audio_codec='aac')
        print(f"Final video with audio saved to {final_output_path}")

    except Exception as e:
        print(f"Error combining audio and video: {e}")


# ---------------------------
# Step 7: Main Execution
# ---------------------------

def main():
    # Define paths
    input_video_path = '/kaggle/working/TalkNet-ASD/demo/upd_trimmed_video/pyavi/video_out.avi'
    audio_path = '/kaggle/working/TalkNet-ASD/demo/extracted_audio.mp3'
    reframed_video_path = '/kaggle/working/TalkNet-ASD/demo/sxsxs_reframed_video.avi'
    final_output_path = '/kaggle/working/TalkNet-ASD/demo/final_output_with_audio.avi'

    # Check if input video exists
    if not os.path.exists(input_video_path):
        print(f"Error: Input video file '{input_video_path}' does not exist.")
        return

    # Determine number of frames from the video
    cap = cv2.VideoCapture(input_video_path)
    if not cap.isOpened():
        print(f"Error: Cannot open video file {input_video_path}")
        return
    num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    cap.release()

    # Extract bounding boxes and scores per frame
    bounding_boxes_per_frame = extract_bounding_boxes(tracks_data, num_frames)
    scores_per_frame = extract_scores_per_frame(tracks_data, scores_data, num_frames)

    # Interpolate missing bounding boxes and scores
    bounding_boxes_per_frame = interpolate_missing_data(bounding_boxes_per_frame, num_frames)
    scores_per_frame = interpolate_missing_data(scores_per_frame, num_frames)

    # Step 1: Extract audio from original video
    extract_audio(input_video_path, audio_path)

    # Step 2: Reframe video
    reframe_video(
        input_video_path,
        reframed_video_path,
        bounding_boxes_per_frame,
        scores_per_frame,
        aspect_ratio=(9, 16)
    )

    # Step 3: Combine audio with reframed video
    combine_audio_video(reframed_video_path, extracted_audio_path, final_output_path)

    # Optional: Clean up intermediate files
    # os.remove(reframed_video_path)
    # os.remove(extracted_audio_path)

if __name__ == "__main__":
    main()
'''

'import cv2\nimport numpy as np\nimport pickle\nimport subprocess\nimport os\nfrom moviepy.editor import VideoFileClip, AudioFileClip\nfrom scipy.interpolate import interp1d\n\n# ---------------------------\n# Step 1: Load Pickle Files\n# ---------------------------\n\ndef load_pickle_file(path):\n    """\n    Loads a pickle file from the specified path.\n\n    Args:\n        path (str): Path to the pickle file.\n\n    Returns:\n        The loaded pickle data.\n    """\n    with open(path, \'rb\') as f:\n        return pickle.load(f)\n\n# Define the paths to the pickle files\n# Replace these paths with the actual paths to your pickle files\ntracks_pkl_path = "/kaggle/working/TalkNet-ASD/demo/upd_trimmed_video/pywork/tracks.pckl"\nscores_pkl_path = "/kaggle/working/TalkNet-ASD/demo/upd_trimmed_video/pywork/scores.pckl"\n\n# Load the pickle files\ntracks_data = load_pickle_file(tracks_pkl_path)\nscores_data = load_pickle_file(scores_pkl_path)\n\n# ---------------------------\n# Step 2: V

In [150]:
from moviepy.editor import VideoFileClip, AudioFileClip
import os
import subprocess

def convert_avi_to_mp4(input_path, output_path):
    """
    Converts an AVI video to MP4 using FFmpeg with additional parameters for better compatibility.

    Args:
        input_path (str): Path to the input AVI video.
        output_path (str): Path to save the converted MP4 video.
    """
    command = [
        'ffmpeg',
        '-y',  # Overwrite output file if it exists
        '-analyzeduration', '100M',  # Increase analyzeduration
        '-probesize', '50M',  # Increase probesize
        '-i', input_path,
        '-c:v', 'libx264',
        '-preset', 'fast',
        '-crf', '23',
        '-c:a', 'aac',
        '-b:a', '192k',
        output_path
    ]
    try:
        subprocess.run(command, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        print(f"Converted {input_path} to {output_path}")
    except subprocess.CalledProcessError as e:
        print(f"Error converting video: {e.stderr.decode()}")

def add_audio_to_reframed_video(reframed_video_path, original_video_path, final_output_path):
    """
    Adds audio from the original video to the reframed video using MoviePy.

    Args:
        reframed_video_path (str): Path to the reframed video file without audio.
        original_video_path (str): Path to the original video file with audio.
        final_output_path (str): Path to save the final video with audio.
    """
    try:
        # Convert AVI to MP4 to ensure compatibility
        converted_video_path = reframed_video_path.replace('.avi', '.mp4')
        convert_avi_to_mp4(reframed_video_path, converted_video_path)

        # Check if files exist
        if not os.path.exists(converted_video_path):
            print(f"Error: Converted video file '{converted_video_path}' does not exist.")
            return

        if not os.path.exists(original_video_path):
            print(f"Error: Original video file '{original_video_path}' does not exist.")
            return

        # Load the converted reframed video without audio
        print("Loading converted reframed video...")
        video_clip = VideoFileClip(converted_video_path)
        
        # Debug: Check fps and duration
        print(f"Video FPS: {video_clip.fps}")
        print(f"Video duration: {video_clip.duration} seconds")

        if video_clip.duration is None or video_clip.duration <= 0:
            raise ValueError(f"Video clip duration is invalid. Please check the video file: {converted_video_path}")

        # Load the audio from the original video
        print("Loading audio from the original video...")
        original_video_clip = VideoFileClip(original_video_path)
        audio_clip = original_video_clip.audio

        if audio_clip is None or audio_clip.duration is None or audio_clip.duration <= 0:
            print("Warning: Audio clip could not be loaded or has no duration. Proceeding without audio.")
            final_clip = video_clip
        else:
            print(f"Audio duration: {audio_clip.duration} seconds")
            # Ensure audio duration matches video duration
            if abs(audio_clip.duration - video_clip.duration) > 0.1:
                print("Adjusting audio duration to match video duration...")
                audio_clip = audio_clip.set_duration(video_clip.duration)
                print(f"Adjusted audio duration: {audio_clip.duration} seconds")
            final_clip = video_clip.set_audio(audio_clip)

        # Verify fps is not None
        fps = video_clip.fps
        if fps is None:
            print("Warning: Video FPS is None. Setting to default FPS of 24.")
            fps = 24  # Default FPS

        # Write the final video
        print("Writing final video with audio...")
        final_clip.write_videofile(
            final_output_path,
            codec='libx264',
            audio_codec='aac',
            temp_audiofile='temp-audio.m4a',
            remove_temp=True,
            fps=fps  # Explicitly set fps
        )
        print(f"Final video with audio saved to {final_output_path}")

    except Exception as e:
        print(f"Error adding audio to reframed video: {e}")

# Example usage
reframed_video_path = '/kaggle/working/TalkNet-ASD/demo/sxsxs_reframed_video.avi'
original_video_path = '/kaggle/working/TalkNet-ASD/demo/upd_trimmed_video.mp4'
final_output_path = '/kaggle/working/TalkNet-ASD/demo/sxsxs_the_output_with_audio.mp4'

add_audio_to_reframed_video(reframed_video_path, original_video_path, final_output_path)


Converted /kaggle/working/TalkNet-ASD/demo/sxsxs_reframed_video.avi to /kaggle/working/TalkNet-ASD/demo/sxsxs_reframed_video.mp4
Loading converted reframed video...
Video FPS: 25.0
Video duration: 82.16 seconds
Loading audio from the original video...
Audio duration: 82.13 seconds
Writing final video with audio...
Moviepy - Building video /kaggle/working/TalkNet-ASD/demo/sxsxs_the_output_with_audio.mp4.
MoviePy - Writing audio in sxsxs_the_output_with_audioTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video /kaggle/working/TalkNet-ASD/demo/sxsxs_the_output_with_audio.mp4

Error adding audio to reframed video: must be real number, not NoneType


In [151]:
!pip install --upgrade moviepy


In [153]:
import subprocess
import os

def convert_avi_to_mp4(input_path, output_path):
    """
    Converts an AVI video to MP4 using FFmpeg.

    Args:
        input_path (str): Path to the input AVI video.
        output_path (str): Path to save the converted MP4 video.
    """
    command = [
        'ffmpeg',
        '-y',  # Overwrite output file if it exists
        '-i', input_path,  # Input file
        '-c:v', 'libx264',  # Video codec
        '-preset', 'fast',  # Encoding speed
        '-crf', '23',  # Quality
        '-c:a', 'aac',  # Audio codec
        '-b:a', '192k',  # Audio bitrate
        output_path  # Output file
    ]
    try:
        subprocess.run(command, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        print(f"Successfully converted '{input_path}' to '{output_path}'")
    except subprocess.CalledProcessError as e:
        print(f"Error converting video: {e.stderr.decode()}")
        raise

# Example usage
reframed_video_path = '/kaggle/working/TalkNet-ASD/demo/sxsxs_reframed_video.avi'
converted_video_path = reframed_video_path.replace('.avi', '.mp4')

convert_avi_to_mp4(reframed_video_path, converted_video_path)


Successfully converted '/kaggle/working/TalkNet-ASD/demo/sxsxs_reframed_video.avi' to '/kaggle/working/TalkNet-ASD/demo/sxsxs_reframed_video.mp4'


In [154]:
def extract_audio(input_video_path, output_audio_path):
    """
    Extracts audio from a video file using FFmpeg.

    Args:
        input_video_path (str): Path to the input video file.
        output_audio_path (str): Path to save the extracted audio file.
    """
    command = [
        'ffmpeg',
        '-y',  # Overwrite output file if it exists
        '-i', input_video_path,  # Input video
        '-vn',  # No video
        '-acodec', 'aac',  # Audio codec
        '-b:a', '192k',  # Audio bitrate
        output_audio_path  # Output audio file
    ]
    try:
        subprocess.run(command, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        print(f"Successfully extracted audio to '{output_audio_path}'")
    except subprocess.CalledProcessError as e:
        print(f"Error extracting audio: {e.stderr.decode()}")
        raise

# Example usage
original_video_path = '/kaggle/working/TalkNet-ASD/demo/upd_trimmed_video.mp4'
extracted_audio_path = original_video_path.replace('.mp4', '_extracted.aac')

extract_audio(original_video_path, extracted_audio_path)


Successfully extracted audio to '/kaggle/working/TalkNet-ASD/demo/upd_trimmed_video_extracted.aac'


In [155]:
def merge_audio_video(video_path, audio_path, output_path):
    """
    Merges audio and video into a single MP4 file using FFmpeg.

    Args:
        video_path (str): Path to the video file (MP4 format).
        audio_path (str): Path to the audio file (AAC format).
        output_path (str): Path to save the merged MP4 file.
    """
    command = [
        'ffmpeg',
        '-y',  # Overwrite output file if it exists
        '-i', video_path,  # Input video
        '-i', audio_path,  # Input audio
        '-c:v', 'copy',  # Copy video stream without re-encoding
        '-c:a', 'aac',  # Audio codec
        '-b:a', '192k',  # Audio bitrate
        '-map', '0:v:0',  # Map video from first input
        '-map', '1:a:0',  # Map audio from second input
        output_path  # Output file
    ]
    try:
        subprocess.run(command, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        print(f"Successfully merged into '{output_path}'")
    except subprocess.CalledProcessError as e:
        print(f"Error merging audio and video: {e.stderr.decode()}")
        raise

# Example usage
final_output_path = '/kaggle/working/TalkNet-ASD/demo/the_most_final_output_with_audio.mp4'

merge_audio_video(converted_video_path, extracted_audio_path, final_output_path)


Successfully merged into '/kaggle/working/TalkNet-ASD/demo/the_most_final_output_with_audio.mp4'
